## Analysis of FFPE Adenocarcinoma Prostate with scanpy

In [1]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
import seaborn as sns
import scanorama
import csv
from sklearn.metrics.pairwise import cosine_distances

import SpatialDE

plt.rcParams['figure.figsize']=(8,8)


%load_ext autoreload
%autoreload 2

In [2]:
#read in data
#this is saved locally so need a different function than downloading from 10X server
ffpe_human_prostate_cancer = sc.read_visium('FFPE_Visium_Human_ProstateCancer',count_file='Visium_FFPE_Human_Prostate_Cancer_filtered_feature_bc_matrix.h5')
ffpe_human_prostate_cancer.var_names_make_unique()

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [3]:
ffpe_human_prostate_cancer

AnnData object with n_obs × n_vars = 4371 × 17943
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [4]:
#Calculate some QC metrics for the data
ffpe_human_prostate_cancer.var["mt"] = ffpe_human_prostate_cancer.var_names.str.startswith("MT")
sc.pp.calculate_qc_metrics(ffpe_human_prostate_cancer, qc_vars=["mt"], inplace=True)

In [5]:
#Visualise some QC metrics for the data
%matplotlib notebook

fig, axs = plt.subplots(1, 4, figsize=(15, 4))
sns.distplot(ffpe_human_prostate_cancer.obs["total_counts"], kde=False, ax=axs[0])
sns.distplot(ffpe_human_prostate_cancer.obs["total_counts"][ffpe_human_prostate_cancer.obs["total_counts"] < 20000], kde=False, bins=40, ax=axs[1])
sns.distplot(ffpe_human_prostate_cancer.obs["n_genes_by_counts"], kde=False, bins=60, ax=axs[2])
sns.distplot(ffpe_human_prostate_cancer.obs["n_genes_by_counts"][ffpe_human_prostate_cancer.obs["n_genes_by_counts"] < 4000], kde=False, bins=60, ax=axs[3])
sc.pl.violin(ffpe_human_prostate_cancer,['pct_counts_mt'])

<IPython.core.display.Javascript object>

/opt/anaconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/opt/anaconda3/envs/scanpy/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'feature_types' as categorical
/opt/anaconda3/envs/scanpy/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categori

<IPython.core.display.Javascript object>

In [6]:
ffpe_human_prostate_cancer

AnnData object with n_obs × n_vars = 4371 × 17943
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'spatial'
    obsm: 'spatial'

In [7]:
#Perform some filtering
sc.pp.filter_cells(ffpe_human_prostate_cancer, min_counts=3000)
print(f'Number of cells after min count filter: {ffpe_human_prostate_cancer.n_obs}')
sc.pp.filter_cells(ffpe_human_prostate_cancer, max_counts=35000)
print(f'Number of cells after max count filter: {ffpe_human_prostate_cancer.n_obs}')
ffpe_human_prostate_cancer = ffpe_human_prostate_cancer[ffpe_human_prostate_cancer.obs["pct_counts_mt"] < 4]
ffpe_human_prostate_cancer = ffpe_human_prostate_cancer[ffpe_human_prostate_cancer.obs["pct_counts_mt"] > 0.5]
print(f"#cells after MT filter: {ffpe_human_prostate_cancer.n_obs}")
#Filter out genes that are detected in less than 10 cells
sc.pp.filter_genes(ffpe_human_prostate_cancer, min_cells=10)
print(f'Number of genes after cell filter: {ffpe_human_prostate_cancer.n_vars}')
sc.pp.filter_cells(ffpe_human_prostate_cancer, min_genes = 3000)
print(f'Number of cells after gene filter: {ffpe_human_prostate_cancer.n_obs}')

Number of cells after min count filter: 4216
Number of cells after max count filter: 4213
#cells after MT filter: 4213
Number of genes after cell filter: 15062
Number of cells after gene filter: 3569


In [8]:
##Do normalisation and find highly variable genes
sc.pp.normalize_total(ffpe_human_prostate_cancer, inplace=True)
sc.pp.log1p(ffpe_human_prostate_cancer)
sc.pp.highly_variable_genes(ffpe_human_prostate_cancer, flavor="seurat", n_top_genes=2000)

In [9]:
#Look at distribution after normalisation
fig, axs = plt.subplots(1, 2, figsize=(15, 4))
sns.distplot(ffpe_human_prostate_cancer.obs["total_counts"], kde=False, ax=axs[0])
sns.distplot(ffpe_human_prostate_cancer.obs["n_genes_by_counts"], kde=False, bins=60, ax=axs[1])

<IPython.core.display.Javascript object>

/opt/anaconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


<AxesSubplot:xlabel='n_genes_by_counts'>

In [10]:
#Perform next steps in cluster identification
sc.pp.pca(ffpe_human_prostate_cancer,n_comps=20)
sc.pp.neighbors(ffpe_human_prostate_cancer)
sc.tl.umap(ffpe_human_prostate_cancer)
sc.tl.leiden(ffpe_human_prostate_cancer, key_added='clusters')

In [11]:
#Do some umap visualisations
plt.rcParams["figure.figsize"] = (4, 4)
sc.pl.umap(ffpe_human_prostate_cancer, color=["total_counts", "n_genes_by_counts", "clusters"], wspace=0.4)

<IPython.core.display.Javascript object>

In [12]:
#Visualise in spatial coordinates
plt.rcParams["figure.figsize"] = (8, 8)
sc.pl.spatial(ffpe_human_prostate_cancer, img_key="hires", color=["total_counts", "n_genes_by_counts"])

<IPython.core.display.Javascript object>

In [13]:
#Visualise cell type clusters by spatial organisation
sc.pl.spatial(ffpe_human_prostate_cancer, img_key="hires", color="clusters", size=1.5)

<IPython.core.display.Javascript object>

In [14]:
#Compute marker genes across all clusters and plot heatmap of the top 10 markers in each cluster
sc.tl.rank_genes_groups(ffpe_human_prostate_cancer, "clusters", method="t-test")
sc.pl.rank_genes_groups_heatmap(ffpe_human_prostate_cancer, groups="5", n_genes=10, groupby="clusters")

categories: 0, 1, 2, etc.
var_group_labels: 5


<IPython.core.display.Javascript object>

### Output necessary files

In [15]:
##write this AnnData object to something readable in Seurat - should be h5ad file
ffpe_human_prostate_cancer.write_h5ad(filename='ffpe_human_prostate_cancer.h5ad', compression=None, compression_opts=None, force_dense=None, as_dense=())

In [ ]:
#Write out normalised dataframe with expression values to file so it can be read back in for this purpose
ffpe_human_prostate_cancer.write_csvs('preprocessed_output/',skip_data=False)

### Find SVGs using Squidpy

In [1]:
##Run moran's I on all genes not just HVGs
sq.gr.spatial_neighbors(ffpe_human_prostate_cancer, coord_type="generic", delaunay=True)
sq.gr.spatial_autocorr(ffpe_human_prostate_cancer, genes=ffpe_human_prostate_cancer.var_names, use_raw=False, mode="moran")
ffpe_human_prostate_cancer.uns["moranI"].head(10)

NameError: name 'sq' is not defined

In [ ]:
moran_I = pd.DataFrame(ffpe_human_prostate_cancer.uns['moranI'].sort_values(ascending=False,by=['I']))

##Sort this so corrected pvalue is above 0.05 and I is above 0.5
moran_I = moran_I[moran_I['pval_norm_fdr_bh'] <=0.05]

##The expected value under Moran's spatial autocorrelation is E(I) = -1/(N-1) where N is number of samples or in this case genes
moran_I = moran_I[moran_I['I'] > 0.1]

In [ ]:
##Write these out to file until figure out how to filter
moran_I.to_csv("ffpe_human_prostate_cancer_squidpy_moranI_results_pval_fdr_bh_0.05_moransI_positive.csv"